# psiCI: a configuration-interaction module using Psi4 with DFT/HF orbitals

Use `psiCI` to perform configuration-interaction (CI) calculations with Psi4 using density-functional theory (DFT) and Hartree-Fock orbitals. The module enable CI calculations using arbitrary configuration space for the CI expansion. It also provides support for building the configuration space associated with common active space (single- and multi-reference CIS(D) and RAS). `psiCI` is a Python class.

README for version 00.01.004.

# 1. Background

We consider an atomic or molecular system with $N$ active electrons, described by the Hamiltonian operator (in atomic units unless otherwise specified)

$$ \hat{\mathcal{H}} = -\sum_{k=1}^{N}{\frac{\Delta_k}{2}}+\sum_{k=1}^{N}{V_\text{ne}({\bf r}_k)} + \sum_{1\leq k < l\leq N}{V_\text{ee}({\bf r}_k-{\bf r}_l)}, \quad\quad\quad (1)$$

where $V_\text{ne}$ is the total electron-nucleus interaction potential, *i.e.*, the atomic or molecular potential, and $V_\text{ee}({\bf r})=-\frac{1}{|{\bf r}|}$ is the Coulomb-repulsion potential between electrons.

The starting point for `psiCI` calculations is a DFT or HF set of spatial orbitals $\{\phi_k\}_k$, obtained after running a restricted ground-state calculation with `Psi4`. For each spatial orbital, we form two spin orbitals $\chi_{k^\prime}({\bf x})=\phi_k({\bf r})\alpha({\omega})$ and $\chi_{k^{\prime\prime}}({\bf x})=\phi_k({\bf r})\beta({\omega})$, where $\alpha$ and $\beta$ designate up and down spins, respectively. Then, selecting a set of $N$ spin-orbital indexes $\vec{k}=\{k_1,k_2,\ldots,k_N\}$, we can form the configuration state associated with their Slater determinant as

$$ |\chi_\vec{k}\rangle = |\chi_{k_1} \chi_{k_2} \ldots \chi_{k_N}\rangle = \frac{1}{\sqrt{N!}}\sum_{\sigma\in S_N}{\text{sgn}(\sigma)\prod_{l=1}^{N}{\chi_{\sigma(k_l)}({\bf x}_{k_l})}}, \quad\quad\quad (2)$$

where $S_N$ is the group of permutations in $[1,N]$. The CI calculation then corresponds to projecting the wave functions and Hamiltonian operator of equation (1) into a basis of configuration states. Specifically, upon calculating the CI matrix with elements $H_{\vec{k}\vec{k}^\prime}=\langle\chi_\vec{k}|\hat{\mathcal{H}}|\chi_{\vec{k}^\prime}\rangle$, its eigen avlues and vectors define the electronic energies and multi-Slater-determinant wave function for the ground and excited state, in the configuration state basis.

# 2. psiCI module

The `psiCI` module provides a suite of functionalities to calculate the CI matrix $H_{\vec{k}\vec{k}^\prime}$, building the configuration-state basis, and analyzing the ground- and excited-state of the CI model. `psiCI` is a Python class.

* **Dependencies:** `psiCI` requires the `numpy`, `scipy`, `itertools`, and `psi4` packages.

**Class at a glance:**
* Attributes: `waveFunction`, `numberElectron`, `configuration`, `display`, `tolerance`
* Methods: `__init__` (constructor), `setConfiguration`, `cleanConfiguration`, `showDocumentation`, `computeCI`, `analyzeSpectrum`
* Miscellaneous methods: `au2ev`, `ev2au`

## 2.1. Attributes

The `psiCI` class defines the following public attributes (we leave out of this documentation a few additional private attributes). Attributes are typically set by the constructor or the `setConfiguration` method.

### 2.1.1. `waveFunction`

`psi4.core.RHF`

* Wave function object returned by `Psi4` at the end of a restricted DFT or HF ground state calculation.
* Set by the constructor.

### 2.1.2. `numberElectron`

`int`

* Number of electrons in the atomic or molecular model.
* Set by the constructor.

### 2.1.3. `configuration`

`numpy.array`

* Configuration-state basis
  * To use use standard configurations like CIS, CISD, or RAS, skip this parameter and use the `setConfiguration` method.
  * For user-defined configuration-state basis, specify the basis as a numpy matrix of spin-orbital indexes
    * Each row of `configuration` specifies the indexes (starting at 1) of the spatial orbitals in `waveFunction` to use in the configuration state, with the sign of the index specifying the spin: positive for up- ($\alpha$) and negative for down-spin ($\beta$) electrons.
    * Each row must contain `numberElectron` unique indexes and define a unique Slater determinant in the configuration basis. `psiCI` does not check for these, and failing to do so will result in erroneous results (or calculations to crash)
    * For instance, `[-1,-2,1,3]` corresponds to the Slater determinant $|\phi_1\beta\ \ \phi_2\beta\ \ \phi_1\alpha\ \ \phi_3\alpha\rangle$, where $\phi_1$ is the deepest spatial orbital in the DFT or HF ground-state calculation.
* Set by the constructor and the `setConfiguration` method.

### 2.1.4. `display`

`bool`

* Whether to display the progress of CI calculations
* Set by the constructor.

### 2.1.5. `tolerance`

`float`
* Threshold for setting CI-matrix elements to zero
  * All CI component with magnitude smaller than `tolerance` are manually set to zero.
  * Set `tolerance` to a negative value to disable thresholding.
* Set by the constructor.

## 2.2 Methods

The `psiCI` class defines the following public methods

### 2.2.1. `__init__` (constructor)

`PCI = psiCI(waveFunction,numberElectron=-1,configuration=numpy.empty(0),display=True,tolerance=1e-10)`

Constructor

**Parameters:**

* `waveFunction : psi4.core.RHF`
  * Wave function object returned by `Psi4` at the end of a restructed DFT or HF ground state calculation. `psiCI` uses it to fetch the spatial orbitals and uspecified electronic-structure properties.
* `numberElectron : int`
  * Number of electrons in the atomic or molecular model.
  * If `numberElectron` is negative, `psiCI` reads it off the input `waveFunction`
* `configuration : numpy.array`
  * Configuration-state basis
  * To use use standard configurations like CIS, CISD, or RAS, skip this parameter and use the `setConfiguration` method.
  * For user-defined configuration-state basis, specify the basis as a numpy matrix of spin-orbital indexes
    * Each row of `configuration` specifies the indexes (starting at 1) of the spatial orbitals in `waveFunction` to use in the configuration state, with the sign of the index specifying the spin: positive for up- ($\alpha$) and negative for down-spin ($\beta$) electrons.
    * Each row must contain `numberElectron` unique indexes and define a unique Slater determinant in the configuration basis. `psiCI` does not check for these, and failing to do so will result in erroneous results (or calculations to crash)
    * For instance, `[-1,-2,1,3]` corresponds to the Slater determinant $|\phi_1\beta\ \ \phi_2\beta\ \ \phi_1\alpha\ \ \phi_3\alpha\rangle$, where $\phi_1$ is the deepest spatial orbital in the DFT or HF ground-state calculation.
* `display : bool`
  * Whether to display the progress of CI calculations
* `tolerance : float`
  * Threshold for setting CI-matrix elements to zero
  * All CI component with magnitude smaller than `tolerance` are manually set to zero.
  * Set `tolerance` to a negative value to disable thresholding.

**Returns:**
* `PCI : psiCI`
  * Initialized `psiCI` object, containing a copy of the `waveFunction` object and other CI-model parameters.

### 2.2.2. `setConfiguration`

Set the configuration-state basis. 

`PCI.setConfiguration(reference=np.empty(0),mode="CIS",active=np.empty(0),frozen=np.empty(0),noDouble=np.empty(0),noEmpty=np.empty(0))`

**Parameters:**

* `reference : numpy.array`
  * For single- and multi-reference CIS and CISD calculation, reference state configuration(s).
  * For RAS calculations, the reference is used to determine the number of electrons in each spin channel and has no other bearing in the determination of the active space (*i.e.*, it may not be included in the configuration state basis).
  * If left empty, a single reference singlet (for even `PCI.numberElectron`) or doublet (odd, with the unparied electron in the up-spin channel) state composed of the `PCI.numberElectron` deepest spin orbitals is usedfor the reference.
  * For user-defined reference state, specify the reference as a numpy vector of spin-orbital indexes.
    * Specify the indexes (starting at 1) of the spatial orbitals in `PCI.waveFunction` to use in the reference state, with the sign of the index specifying the spin: positive for up- ($\alpha$) and negative for down-spin ($\beta$) electrons.
    * For instance, `[-1,-2,1,2]` corresponds to the reference Slater determinant $|\phi_1\beta\ \ \phi_2\beta\ \ \phi_1\alpha\ \ \phi_2\alpha\rangle$, where $\phi_1$ is the deepest spatial orbital in the DFT or HF ground-state calculation.
    * For multi-reference CIS(D), specify the different references in separate rows of a matrix; for instance `[[-1, -2, 1, 2],[-1, -3, 1, 3]]`.
    * When a `reference` is specified, the `PCI.numberElectron` is updated accordingly.
* `mode : str`
  * Flavor of CI for which to set the configuration-state basis
  * `"CIS"` sets the configuration-state basis for single-excitation type CI (CIS).
    * Single- vs multi-reference CIS is determined by the shape of `reference`
  * `"CISD"` sets the configuration-state basis for single- and double-excitation type CI (CISD).
    * Single- vs multi-reference CIS is determined by the shape of `reference`
  *  `"RAS"` sets the configuration basis for restricted active space type CI (RAS).
    * If no `noDouble` or `noEmpty` are specified, this corresponds to a complete active space (CAS) model.
* `active : numpy.array`
  * Indexes of the spin orbitals to use in the configuration space expansion
  * If left empty, all the spin orbitals in `PCI.waveFunction` are included in the configuration basis, thus corresponding to full CIS(D)/RAS/CAS.
  * For user-defined active space, specify the active space as a numpy vector of spin-orbital indexes
    * Specify the indexes (starting at 1) of the spatial orbitals in `PCI.waveFunction` to use in the active space, with the sign of the index specifying the spin: positive for up- ($\alpha$) and negative for down-spin ($\beta$) electrons.
    * For instance, `[-1,-2,-3,-5,1,2,3,5]` builds the active space for CIS(D) including excitations involding both the up- ($\alpha$) and down-spin ($\beta$) orbitals with spatial components $\phi_1$, $\phi_2$, $\phi_3$, $\phi_5$ and skipping $\phi_4$.
  * For multi-reference CIS(D) calculations, all the references share the same active space
* `frozen : numpy.array`
  * Indexes of the frozen spin orbitals, which must contain one electron. 
  * If left empty, no orbitals are frozen.
  * Specify the indexes (starting at 1) of the spatial orbitals in `PCI.waveFunction` to freeze in the active space, with the sign of the index specifying the spin: positive for up- ($\alpha$) and negative for down-spin ($\beta$) electrons.
    * For instance, `[-1, 1, 2]` indicates all configuration states in the configuration state basis must include the spin orbitals with spatial components $\phi_1$, $\phi_2$.
  * For single- and multi-reference CIS(D) calculations, only frozen orbitals that are in the reference are considered.
  * For RAS calculations, the number of frozen orbitals in each spin channel must be compatible with the number of electrons each hold (defined via the `reference`).
* `noDouble : numpy.array`
  * Indexes of the **spatial** orbitals that may not contain two electrons, *i.e.*, cannot have an electron in both the up and down spin channels.
  * If left empty, no restriction on double occupation is imposed.
  * Specify the indexes (starting at 1) of the spatial orbitals in `PCI.waveFunction` that may not contain double excitations
    * For instance, `[5, 6]` indicates that the orbitals with spatial components $\phi_5$ and $\phi_6$ may not be fully occupied.
  * For single- and multi-reference CIS(D), the `noDouble` constraint is **not** imposed on the `reference` configuration(s).
* `noEmpty : numpy.array`
  * Indexes of the **spatial** orbitals that may not be left empty, *i.e.*, must have at least one electron in the up or down spin channel.
  * If left empty, no restriction on empty occupation is imposed.
  * Specify the indexes (starting at 1) of the spatial orbitals in `PCI.waveFunction` that may not be empty.
    * For instance, `[1, 2]` indicates that the orbitals with spatial components $\phi_1$ and $\phi_2$ should always hold at least one electron.
  * For single- and multi-reference CIS(D), the `noEmpty` constraint is **not** imposed on the `reference` configuration(s).

**Returns:**

* N/A: the configuration-state basis is stored in the `psiCI` object `PCI`.

### 2.2.3. `cleanConfiguration`

Remove duplicate states in the configuration state basis.

`cleanConfiguration()`

* Use cleanConfiguration to remove duplicate configuration states in configuration, including those related by a permutation of the spin orbitals in the index vector.
* The cleaning process preserves the order of (i) configuration states in the basis, keeping only the first unique instance of each state, and (ii) the order of the spin-orbital withing each configuration state.
  * For instance, the configuration `[[-1, -2, 1, 2],[-1, -3, 1, 2],[-2, -1, 1, 2],[-1, -2, 1, 3]]` is cleaned to `[[-1, -2, 1, 2],[-1, -3, 1, 2],[-1, -2, 1, 3]]` since `[-2, -1, 1, 2]` is a duplicate of `[-1, -2, 1, 2]`, and thus removed from the set.

**Returns:**

* N/A: the cleaning is performed directly in `PCI.configuration`.

### 2.2.4. `showDocumentation`

Display the run-time documentation

`PCI.showDocumentation()`

* Prints out the configuration of the CI model associated with the `PCI` object.

### 2.2.5. `computeCI`

Compute the CI matrix

`CI = PCI.computeCI(returnDipole=False)`

`CI, Dx, Dy, Dz = PCI.computeCI(returnDipole=True)`

**Parameters:**

* `returnDipole : bool`
  * Whether to calculate and return the dipole transition matrices in the x, y, and z directions alongside the CI matrix
  * Empty or `False` disables the dipole calculation and only returns the CI matrix

**Returns:**

* `CI : numpy.array`
  * CI matrix with elements $H_{kl}=\langle\chi_\vec{k}|\hat{\mathcal{H}}|\chi_{\vec{l}^\prime}\rangle$ where $|\chi_{\vec{k,l}^\prime}\rangle$ are the configuration states defined by `PCI.configuration[k,:]` and `PCI.configuration[l,:]`, respectively.
  * **Note:** the CI matrix only contains the electronic part of the molecular energy. To get the nuclear-repulsion part use the `nuclear_repulsion_energy()` method.
* `Dx : numpy.array`
  * Dipole-transition matrix in the x direction; only returned when `returnDipole` is `True`
* `Dy : numpy.array`
  * Dipole-transition matrix in the y direction; only returned when `returnDipole` is `True`
* `Dz : numpy.array`
  * Dipole-transition matrix in the z direction; only returned when `returnDipole` is `True`

### 2.2.6. `analyzeSpectrum`

Analyze the spectrum of a CI matrix

`PCI.analyzeSpectrum(CI,state=numpy.empty(0),tolerance=1e-2)`

**Parameters:**

* `CI : numpy.array`
  * CI matrix, typically obtained from the `computeCI` method
  * The CI matrix must be real symmetric.
* `state : numpy.array`
  * Index(es) of the excited states for which to display the analysis, starting at 1 for the first excited state
    * For each excited state, the analysis reports the total and escitation enerfies (compared to the ground state; see next), as well as the make up of the state
    * For instance `[1, 2, 5]` displays the analysis results for the first, second, and fifth excited states, skipping the third and fourth.
  * If left empty, the spectrum analysis only reports the energy and makeup of the ground state (eigen vector of the CI matrix with smallest energy)
* `tolerance : float`
  * Population threshold: For the ground and each of the requested excited states, only configuration states that contribute a population larger than `tolerance` to the wave function are listed in the display.

**Returns:**

* N/A: `analyzeSpectrum` prints out the result of the analysis
  * The population makeup reports the population contribution (amplitude squared) and configuration state, as defined in the `configuration` property: a vector of spin-orbital indexes.
 
## 2.3. Miscellaneous methods

`psiCI` also provides miscellaneous methods for unit conversions.

### 2.3.1. `au2ev`

Convert energy from atomic units to electron volts, using the conversion 1 \[a.u.\] = 27.211386245988 \[eV\]

`evValue = psiCI.au2ev(auValue)`

**Parameters:**
* `auValue : float or numpy.array`
  * Atomic unit energy(ies) to be converted to electron volts
 
**Returns:**
* `evValue : float or numpy.array`
  * Converted energy(ies) in electron volts
 
### 2.3.1. `ev2au`

Convert energy from electron volts to atomic units, using the conversion 1 \[eV\] = 1/27.211386245988 \[a.u.\]

`auValue = psiCI.ev2au(auValue)`

**Parameters:**
* `evValue : float or numpy.array`
  * Electron volt energy(ies) to be converted to atomic units
 
**Returns:**
* `auValue : float or numpy.array`
  * Converted energy(ies) in atomic units

# 3. Examples

All examples require importing the `numpy` and `psi4` packages

In [1]:
import numpy as np
import psi4
import psiCI

## 3.1. LiH molecule

In this example, we consider the LiH molecule. A preliminary step for the CI calculation is to define the molecular model and calculate its ground state, here taken at the restricted HF level

In [2]:
# Molecular model
LiH = psi4.geometry("""
         0 1
         H   0.865 0 0
         Li -0.865 0 0
         symmetry c1 """) # force no symmetry

# HF ground state
psi4.set_options({
        'basis':        'sto-3g',
        'scf_type':     'pk',
        'reference':    'rhf',
        'mp2_type':     'conv',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8})

scf_e, wfn = psi4.energy('scf', return_wfn=True)

We can now define a `psiCI` object to define our CI model. At minimum, we must specify the `Psi4` wave function and leave all other parameters to their default values.

In [3]:
PCI = psiCI.psiCI(wfn)

Next, we set the configuration state basis. Here we aim to perform CIS calculation involving the first 4 orbitals in each spin channel and let `psiCI` retrieve the reference state configuration

In [4]:
# Active spin-orbital basis
MO = np.asarray(range(1,5))
SO = np.concatenate((-MO,MO))

# Set the configuration-state basis
PCI.setConfiguration(mode="CIS",active=SO)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 4
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Active      = [-1 -2 -3 -4]
                = [1 2 3 4]
  * 9 configurations

################################################################################



Since the whave left `PCI.display` to its default `True` value, the configuration-state basis builder prints out a summary of the parameters and output results. We can check the overall configuration of the `PCI` object using the run-time documentation method

In [5]:
PCI.showDocumentation()

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Electronic structure =======================================================
  * 4 electrons
  * 6 spatial orbitals
  * 9 configurations

=== Configuration-interaction calculation ======================================
  * tolerance: 1e-10

################################################################################



Note that, the run-time documentation indicates the number of spatial orbitals in the `waveFunction` object, not the number of orbitals included configuration-state basis. We now have the `PCI` object fully ready to calculate the CI matrix

In [6]:
CI = PCI.computeCI()

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Electronic structure =======================================================
  * 4 electrons
  * 6 spatial orbitals
  * 9 configurations

=== Configuration-interaction calculation ======================================
  * tolerance: 1e-10
  * 4 active spatial orbitals

=== CI matrix calculation ======================================================
  * Core Hamiltonian                                                        done
  * Two-electron integrals                                                  done
  * CI matrix                                                               done

######################################################

As expected, the CI calculation includes a reduced number (4) of active spatial orbitals in the calculation, compared to the available ones (6) in the `waveFunction` object.

In [7]:
# Excited states to include in the analysis
ES = np.array([1,2,3,6])

# Display spectrum analysis result
PCI.analyzeSpectrum(CI,state=ES)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== CI matrix spectrum =========================================================
  * Ground state
    Total energy      =     -8.773 a.u. =   -238.722 eV
    > 100.000 % [-1 -2  1  2]

  * Excited state 1
    Total energy      =     -8.642 a.u. =   -235.168 eV
    Excitation energy =      0.131 a.u. =      3.554 eV
    >  49.999 % [-1 -3  1  2]
    >  49.999 % [-1 -2  1  3]

  * Excited state 2
    Total energy      =     -8.613 a.u. =   -234.378 eV
    Excitation energy =      0.160 a.u. =      4.344 eV
    >  49.998 % [-1 -3  1  2]
    >  49.998 % [-1 -2  1  3]

  * Excited state 3
    Total energy      =     -8.599 a.u. =   -233.990 eV
    Exc

As expected, we see that the CIS ground-state wave function is the same as the HF one, with 100% of the reference configuration state.

Next, we repeat the CI calculation, this time including double excitations in the active space and also requesting the dipole coulpin matrices.

In [8]:
# Update the configuration-state basis
PCI.setConfiguration(mode="CISD",active=SO)

# Recalculate the CI matrix and dipole coupling
CI, Dx, Dy, Dz = PCI.computeCI(returnDipole=True)

# Display spectrum analysis result
PCI.analyzeSpectrum(CI,state=ES)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Nb. elec.   = 4
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Active      = [-1 -2 -3 -4]
                = [1 2 3 4]
  * 27 configurations

################################################################################

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.

## 3.2. Tailoring the configuration state basis

In this example, we consider the N$_2$ molecule to illustrate how one can use the `setConfiguration` to tailor the configuration state basis.

To begin with, we calculate the molecular orbitals basis using DFT with the PBE0 exchange-correlation functional.

In [9]:
# Import proper packages
import psi4
import numpy as np
import psiCI

# Molecular model
N2 = psi4.geometry("""
        0 1
          N  0.55 0 0
          N -0.55 0 0
        symmetry c1 """)

# psi4 parameters
psi4.set_options({
        'basis':        'cc-pvtz',
        'scf_type':     'pk',
        'reference':    'rhf',
        'mp2_type':     'conv',
        'e_convergence': 1e-10,
        'd_convergence': 1e-10})

# DFT ground-state orbitals
scf_e, wfn = psi4.energy('pbe0', return_wfn=True)

# Show the DFT-orbital energies
E = psiCI.au2ev(wfn.epsilon_a().np)
print("Occupied orbital energies (eV):")
for k in range(7):
     print("  > orbital " + str(k+1) + " = " + str(E[k]))

print("\nVirtual orbital energies (ev)")
for k in range(7,12):
     print("  > orbital " + str(k+1) + " = " + str(E[k]))

print(" ")

Occupied orbital energies (eV):
  > orbital 1 = -394.1757601787032
  > orbital 2 = -394.1252941904906
  > orbital 3 = -31.347769526223562
  > orbital 4 = -15.574657396358754
  > orbital 5 = -13.059052447993846
  > orbital 6 = -13.059052447993825
  > orbital 7 = -12.077610163966229

Virtual orbital energies (ev)
  > orbital 8 = -0.42660408542233436
  > orbital 9 = -0.42660408542228545
  > orbital 10 = 8.436996864729764
  > orbital 11 = 11.138265262311707
  > orbital 12 = 11.13826526231182
 


Wee see that the occupied spatial orbitals 5 through 7 have similar energies, and that orbitals 8 and 9 have a notably lower energy as the other virtual orbitals. To identify which refernece(s) we should include in our CI calculations, we first perform a CAS-level calculation including the spatial orbitals 5 through 9 in the active space (and orbitals 1 through 4 are frozen).

In [10]:
# Create the CI object
PCI = psiCI.psiCI(wfn)
PCI.showDocumentation()

# Build the configuration state basis
MO  = np.asanyarray(range(1,10))
FRZ = np.asanyarray(range(1,5))
PCI.setConfiguration(mode="RAS",active=np.concatenate((-MO,MO)),frozen=np.concatenate((-FRZ,FRZ)))

# Calculate the CI matrix
CI = PCI.computeCI()

# Display spectrum analysis result
PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Electronic structure =======================================================
  * 14 electrons
  * 60 spatial orbitals
 
=== Configuration-interaction calculation ======================================
  * tolerance: 1e-10

################################################################################

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025


We now use these results to define our extended configuration basis, using a multi-reference CISD model, as follows
* Three configurations have populations larger than 1% and use all of them as a reference.
* Orbitals 1 and 2 correspond to core electrons and we freeze them.
* Orbital 3 corresponds to inner valence electrons and impose that all configuration must have at least one electron in it (not empty).
* We include all the orbitals with DFT energy lower smaller than 25 eV in the active space.

In [11]:
# Configuration state basis parameters
ref = np.array([[-4, -3, -2, -1, -7, -6, -5, 1, 2, 3, 4, 5, 6, 7],
                [-4, -3, -2, -1, -8, -7, -5, 1, 2, 3, 4, 5, 7, 8],
                [-4, -3, -2, -1, -9, -7, -6, 1, 2, 3, 4, 6, 7, 9]])
frz = np.array([-1, -2, 1, 2])
noEmpt = np.array([3])
MO = np.asarray(range(1,sum(E <= 25)+1)) # E is already in eV
act = np.concatenate((-MO,MO))

# Build the configuration-state basis
PCI.setConfiguration(mode="CISD",reference=ref,active=act,frozen=frz,noEmpty=noEmpt)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Nb. elec.   = 14
  * Total spin  = [0. 0. 0.]
  * References  = [-4 -3 -2 -1 -7 -6 -5  1  2  3  4  5  6  7]
                  [-4 -3 -2 -1 -8 -7 -5  1  2  3  4  5  7  8]
                  [-4 -3 -2 -1 -9 -7 -6  1  2  3  4  6  7  9]
  * Active      = [ -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17 -18
 -19 -20]
                = [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
  * Frozen      = [-1 -2  1  2]
  * Not empty   = [3]
  * 16206 configurations

#####################################################

In [12]:
# Calculate the new ground state
CI = PCI.computeCI()

# Display spectrum analysis result
PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Electronic structure =======================================================
  * 14 electrons
  * 60 spatial orbitals
  * 16206 configurations

=== Configuration-interaction calculation ======================================
  * tolerance: 1e-10
  * 20 active spatial orbitals

=== CI matrix calculation ======================================================
  * Core Hamiltonian                                                        done
  * Two-electron integrals                                                  done
  * CI matrix                |||||||||||||||||||||||||||||||||||||||||||||||||||

###############################################

One may also build customized configuration basis sets by combining several CIS(D)/RAS ones. For instance, in the following example using the same 3 references we add single excitations to virtual orbitals up to 35 eV.

In [13]:
# CISD basis (up to 25 eV)
MO = np.asarray(range(1,sum(E <= 25)+1))
act = np.concatenate((-MO,MO))
PCI.setConfiguration(mode="CISD",reference=ref,active=act,frozen=frz,noEmpty=noEmpt)
CISD = PCI.configuration.copy()

# CIS basis (up to 35 eV)
MO = np.asarray(range(1,sum(E <= 35)+1))
act = np.concatenate((-MO,MO))
PCI.setConfiguration(mode="CIS",reference=ref,active=act,frozen=frz,noEmpty=noEmpt)
CIS = PCI.configuration.copy()

# Copbine the bases and remove duplicates
PCI.configuration = np.concatenate((CISD,CIS),axis=0)
PCI.cleanConfiguration()

#Calculate the new ground state
CI = PCI.computeCI()

# Display spectrum analysis result
PCI.analyzeSpectrum(CI)

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Nb. elec.   = 14
  * Total spin  = [0. 0. 0.]
  * References  = [-4 -3 -2 -1 -7 -6 -5  1  2  3  4  5  6  7]
                  [-4 -3 -2 -1 -8 -7 -5  1  2  3  4  5  7  8]
                  [-4 -3 -2 -1 -9 -7 -6  1  2  3  4  6  7  9]
  * Active      = [ -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17 -18
 -19 -20]
                = [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
  * Frozen      = [-1 -2  1  2]
  * Not empty   = [3]
  * 16206 configurations

#####################################################